# Preprocessign data sets

## 1. Clean Up Pavlopoulos(2020)
- 공개 데이터셋, Pavlopoulos et al,. 2020. Toxicity Detection: Does Context Really Matter?.
- 이모지, URL, \t,\s와 같은 tag 만 제거하였음.

In [ ]:
!wget https://raw.githubusercontent.com/ipavlopoulos/context_toxicity/master/data/CAT_LARGE/gc.csv gc.csv

import pandas as pd
import re

--2021-07-13 04:53:24--  https://raw.githubusercontent.com/ipavlopoulos/context_toxicity/master/data/CAT_LARGE/gc.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3697430 (3.5M) [text/plain]
Saving to: ‘gc.csv’

gc.csv              100%[===================>]   3.53M  14.4MB/s    in 0.2s    

2021-07-13 04:53:24 (14.4 MB/s) - ‘gc.csv’ saved [3697430/3697430]

--2021-07-13 04:53:24--  http://gc.csv/
Resolving gc.csv (gc.csv)... failed: Name or service not known.
wget: unable to resolve host address ‘gc.csv’
FINISHED --2021-07-13 04:53:24--
Total wall clock time: 0.7s
Downloaded: 1 files, 3.5M in 0.2s (14.4 MB/s)


In [ ]:
df = pd.read_csv('gc.csv')
df.head()

,id,text,parent,label,api
0,100030037.40314.40314,"BTW, I see no ""attack"" here. What I see is so...",I also feel its important for StuRat to have h...,0,0.1323
1,100104685.102437.102437,I realise... I just want to urge caution and a...,A main reason this project was started was to ...,0,0.0314
2,100189373.112824.112824,I should have stressed that this should be a t...,"I'd go for that. (By the way, not all deletion...",0,0.2229
3,100203140.40393.40393,Thanks to Lysy for shortening and copy editing...,(edit conflict with Piotrus) I agree with the...,0,0.0559
4,100214962.61084.61084,I like the additional info in the info box. I...,The larger box displays fine in Mozilla Firefo...,0,0.0539


In [ ]:
df.label.value_counts()

0    9849
1     151
Name: label, dtype: int64

In [ ]:
def remove_emoji_gc(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    string = emoji_pattern.sub(r'', string)

    comment_number = re.compile("[0-9]+.")
    string = comment_number.sub(r'',string)
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    # url = re.compile('((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*')
    url = re.compile(regex)
    string = url.sub(r'',string)
    string = re.compile("[']{2,3}").sub(r"'",string)
    string = re.compile("[\'|]{1}").sub(r"'",string)
    string = re.compile("[\']{2,}").sub(r"",string)
    string = re.compile(r"\s+").sub(" ", string).strip()
    return string

In [ ]:
for n, (text, parent, label) in enumerate(zip(df['text'], df['parent'], df['label'])):
  parent = remove_emoji_gc(parent)
  text = remove_emoji_gc(text)
  print(f'Q {n} : {repr(parent)}\nA {n} : {repr(text)}\n L {label}\n')

In [ ]:
for n, (text, parent) in enumerate(zip(df[df['label'] == 1]['text'], df[df['label']==1]['parent'])):
  print(f'Q {n} : {parent}\nA {n} : {text}\n')

In [ ]:
df['text_clean'] = df['text'].apply(remove_emoji_gc)
df['parent_clean'] = df['parent'].apply(remove_emoji_gc)

df.head()

,id,text,parent,label,api,text_clean,parent_clean
0,100030037.40314.40314,"BTW, I see no ""attack"" here. What I see is so...",I also feel its important for StuRat to have h...,0,0.1323,"BTW, I see no ""attack"" here. What I see is som...",I also feel its important for StuRat to have h...
1,100104685.102437.102437,I realise... I just want to urge caution and a...,A main reason this project was started was to ...,0,0.0314,I realise... I just want to urge caution and a...,A main reason this project was started was to ...
2,100189373.112824.112824,I should have stressed that this should be a t...,"I'd go for that. (By the way, not all deletion...",0,0.2229,I should have stressed that this should be a t...,"I'd go for that. (By the way, not all deletion..."
3,100203140.40393.40393,Thanks to Lysy for shortening and copy editing...,(edit conflict with Piotrus) I agree with the...,0,0.0559,Thanks to Lysy for shortening and copy editing...,(edit conflict with Piotrus) I agree with the ...
4,100214962.61084.61084,I like the additional info in the info box. I...,The larger box displays fine in Mozilla Firefo...,0,0.0539,I like the additional info in the info box. It...,The larger box displays fine in Mozilla Firefo...


In [ ]:
for n, (text, parent) in enumerate(zip(df[df['label'] == 1]['text_clean'], df[df['label']==1]['parent_clean'])):
  print(f'Q {n} : {parent}\nA {n} : {text}\n')

In [ ]:
df_clean = df[['parent_clean','text_clean','label']]
df_clean.rename({'parent_clean':'parent','text_clean':'text'},axis=1,inplace=True)
df_clean.to_csv('gc_clean.csv')


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


### NER by spacy
- fine-tuning GPT Neo 활용 문장 생성시, NER을 Keyword로 사용하려는 계획.(시간 부족으로 실제 문장 생성은 하지 못함)

In [ ]:
!pip install -U spacy
!python -m spacy download en_core_web_trf

In [ ]:
import spacy
from spacy.tokenizer import Tokenizer


In [ ]:
nlp = spacy.load("en_core_web_trf", disable=["tagger", "parser", "attribute_ruler", "lemmatizer"])

def ner(text):
  doc = nlp(text)
  for ent in doc.ents:
    if ent:
      return ','.join([ent.text, ent.label_])
    else :
      return 'None'


In [ ]:
df_clean['parent_ner'] = df_clean['parent'].apply(ner)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df_clean['text_ner'] = df_clean['text'].apply(ner)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df_clean.to_csv('gc_clean_ner.csv')

In [ ]:
from google.colab import files
files.download('gc_clean_ner.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
files.download('reddit.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


## Clean up Qian(2019), Gab & Reddit data
- Pavlopoulos(2020) 데이터와 유사하게 전처리 수행하였으며. 추가로 @tag로 user를 tag한 부분은 삭제, #tag keyword 부분은 남겨두었음.


In [ ]:
!wget https://raw.githubusercontent.com/jing-qian/A-Benchmark-Dataset-for-Learning-to-Intervene-in-Online-Hate-Speech/master/data/gab.csv gab.csv
!wget https://raw.githubusercontent.com/jing-qian/A-Benchmark-Dataset-for-Learning-to-Intervene-in-Online-Hate-Speech/master/data/reddit.csv raddit.csv

import numpy as np
import pandas as pd
import re

def check_text(df,ranges):
  for i in range(ranges[0],ranges[1]):
    print(df.iloc[i,0])
    print(str(df.iloc[i,1]))
    print(df.iloc[i,2])
    print(df.iloc[i,3])
    print('\n\n')

In [4]:
class Regex_pattern():
  def __init__(self):
    self.emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    self.comment_number = re.compile("[0-9]+.")
    self.url = re.compile("(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))")
    self.user = re.compile(r"@[a-zA-Z]+")
    self.hash = re.compile(r"#")


def remove_emoji(string, regex_pattern):

    string = regex_pattern.emoji_pattern.sub(r'', string)
    string = regex_pattern.comment_number.sub(r'',string)
    string = regex_pattern.url.sub(r'',string)
    # user @ tag는 전체 삭제
    string = regex_pattern.user.sub(r'',string)
    # #tag 해시태그는 #해시 문자만 삭제
    string = regex_pattern.hash.sub(r'',string)
    # string = re.compile(r"#[a-zA-Z]+").sub(r'',string)
    return string


In [5]:
regex_pattern = Regex_pattern()

In [6]:
print(remove_emoji(df_rg.loc[692,'text'], regex_pattern))

NameError: ignored

In [ ]:
df_gab = pd.read_csv('gab.csv')
df_reddit = pd.read_csv('reddit.csv')

df_rg = pd.concat([df_gab,df_reddit], ignore_index=True)
# df_rg.dropna(inplace=True) # nan for non-toxic label
print(df_rg.isnull().sum(),df_rg.shape)
df_rg.duplicated().sum()

# context가 있는 comment만 남김
df_rg = df_rg[~(df_rg.hate_speech_idx == '[1]')][['text','hate_speech_idx']]
df_rg.head()
df_rg.shape

In [ ]:
for i,j in zip(df_rg['text'][700:750], df_rg['hate_speech_idx']):
  print(f'\n{j}\n{i}\n\n')

### Qian(2019) 데이터 문장 분리 로직
- 2차례 이상 이어지는 User간 Conversation 중, 혐오 발언으로 간주될 수 있는 Comment 번호를 라벨링 하였음(Figure 2의 예시 참고).
- Hate Speech로 label된 문장을 중심으로, 그 이전 Comment를 맥락 text로 추출하였음.
- 하나의 Conversation에 여러개의 Hate Speech로 label이 있는 경우, Hate Speech label 수만큼 분리.
- Hate Speech로 label된 문장 이전 맥락이 지나치게 짧은 경우(예를 들어 한단어), 맥락 정보가 포함되지 않는 문장이 되어버리기 때문에, 적어도 5개 단어 토큰(stop word 제외) 보다 커질 때 까지 이전 comment를 포함시킴.

In [ ]:
!pip install -U spacy
!python -m spacy download en_core_web_trf

import spacy
from spacy.tokenizer import Tokenizer

# 맥락 commnet의 token을 카운트 할 때, stop word, punct를 걸러내기 위해서 사용
nlp = spacy.load("en_core_web_trf")
tokenizer = Tokenizer(nlp.vocab)

In [ ]:
def token_count(doc):
  cnt = 0
  for token in tokenizer(doc): # spacy tokenizer
    # print(t)
    if (token.is_stop == False) & (token.is_punct == False):
      cnt +=1
  return cnt

def concat_doc(doc_list):
  if len(doc_list) < 2:
    return doc_list
  else :
    return ['. '.join(reversed(doc_list))]

# def set_text_parent(comments, idx):
def set_text_parent(x, regex_pattern):
  comments = x[0]
  idx = x[1]

  comments = remove_emoji(comments, regex_pattern)
  comments = comments.replace('\t','').split('\n')

  doc = []
  for i in comments:
    i = re.compile(r"\s+").sub(" ", i).strip() # too many space -> single space
    doc.append(i)

  doc = doc[:-1]

  text = []
  parent = []
  label = []

  # for non-toxic comments
  if str(idx) == 'nan':
    text.append(doc[-1])
    max_doc = len(doc)
    cnt = 0
    for i in range(max_doc-2, -1,-1):
      cnt += token_count(doc[i])
      parent.append(doc[i])
      label.append(0)
      if cnt > 5:
        return concat_doc(parent), text, label
      else : continue
    return concat_doc(parent), text, label

  # set start idx 0
  idx_list = [int(i)-1 for i in re.compile(r'\d{1,2}').findall(idx)]

  # only single comment is toxic & single context commnet
  if idx_list[0] == 1 and len(idx_list) < 2:
    parent.append(doc[idx_list[0]-1])
    text.append(doc[idx_list[0]])
    label.append(1)
    return parent, text, label # return single string sentence list

  # single toxic
  if len(idx_list) < 2:
    text.append(doc[idx_list[-1]])
    label = [1]
    cnt = 0
    for i in range(idx_list[-1]-1, -1, -1): 

      cnt += token_count(doc[i])
      parent.append(doc[i])
      
      if cnt > 5:
        return concat_doc(parent), text, label
      else : continue
    return concat_doc(parent), text, label


  if len(idx_list) > 1:

    for j in idx_list:
      if j == 0:
        continue
      
      text.append(doc[j])
      label = [1]
      cnt = 0
      tmp_parent = []
      for i in range(j-1,-1,-1):
        cnt += token_count(doc[i])
        tmp_parent.append(doc[i])
        if cnt > 5:
          parent.append(concat_doc(tmp_parent)[0])
          break
        else : continue
        parent.append(concat_doc(tmp_parent)[0])

  return parent, text, label


In [ ]:
df_rg.head()

In [ ]:
df_rg.iloc[10612,0]

In [ ]:
from tqdm.notebook import tqdm

t_p, t_t, t_l = [], [], []

for n, (c,i) in enumerate(zip(df_rg['text'],df_rg['hate_speech_idx'])):
  if n % 1000 == 0:
    print(n)
  tmp = (c,i)
  try :
    p, t, l = set_text_parent(tmp, regex_pattern)
  except :
    print(n)
  # print(p,t,l)
  for sp,st in zip(p, t):
    t_p.append(sp)
    t_t.append(st)
    # print(l)
    t_l.append(l[0])
    # print(t_p,t_t,l)

# for n,(p, t) in enumerate(zip(t_p, t_t)):
  # print(f'{n}\nparent - {p}\ntext - {t}\n')

df_split = pd.DataFrame(list(zip(t_p,t_t,t_l)), columns=['parent','text','label'])

In [ ]:
df_split

In [ ]:
df_split.label.value_counts()

In [ ]:
df_split.to_csv('gab_reddit.csv')
from google.colab import files
files.download('gab_reddit.csv') 

In [ ]:
df_split[df_split.label == 0]

### NER by spacy

In [ ]:
# nlp = spacy.load('en_core_web_trf')
nlp = spacy.load("en_core_web_trf", disable=["tagger", "parser", "attribute_ruler", "lemmatizer"])

def ner(text):
  doc = nlp(text)
  for ent in doc.ents:
    if ent:
      return ','.join([ent.text, ent.label_])
    else :
      return 'None'

df_split['parent_ner'] = df_split['parent'].apply(ner)
df_split['text_ner'] = df_split['text'].apply(ner)

df_split.to_csv('gab_reddit_ner.csv')
from google.colab import files
files.download('gab_reddit_ner.csv') 

In [ ]:
df_split.label.value_counts()

In [ ]:
for n, (c, i) in enumerate(zip(df_rg['text'][450:500],df_rg['hate_speech_idx'][450:500])):
  print(n,'\n',c, i,'\n')

# Prepare Training Data with Generated Data by GPT-neo
- Pavlopoulos, Qian 데이터를 기반으로 GPT neo를 fine-tuning 하였고. 문장을 생성하였음.
- 시간 절약을 위해 parent, text 각각 동시에 튜닝 & 문장생성 하였음.
- 개별 csv data file을 통합 하였음. 
```
맥락 문장 = parent
혐오 문장 = text
```

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

In [ ]:
class MyConfig(dict):
  def __getattr__(self, name): return self[name]
  def __setattr__(self, name, value): self[name] = value

configs = MyConfig({'path':{
    'path_drive' : Path('/content/gdrive/MyDrive/toxic_nlp/dataset'),
    'path_colab_data' : Path('./dataset')
    }
})

In [ ]:
configs.path['list_csv'] = [f for f in configs.path['path_drive'].glob('*.csv')]
configs

{'path': {'list_csv': [PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/gc_clean.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/gc_clean_ner.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/gab_reddit.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/gab_reddit_ner.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/parent_gen_data.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/text_gen_data.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/generated_dataset.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/parent_gen_data_v2.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/text_gen_data_v2.csv')],
  'path_colab_data': PosixPath('dataset'),
  'path_drive': PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset')}}

In [ ]:
df_p = pd.read_csv('/content/gdrive/MyDrive/toxic_nlp/dataset/parent_gen_data_v2.csv', index_col=0)
df_t = pd.read_csv('/content/gdrive/MyDrive/toxic_nlp/dataset/text_gen_data_v2.csv', index_col=0)

In [ ]:
df_p.head()

,parent,text,label,parent_gen
0,I also feel its important for StuRat to have h...,"BTW, I see no ""attack"" here. What I see is som...",0,I also feel its important for StuRat to have ...
1,A main reason this project was started was to ...,I realise... I just want to urge caution and a...,0,A main reason this project was started was to...
2,"I'd go for that. (By the way, not all deletion...",I should have stressed that this should be a t...,0,I'd go for that. A lot of the most common are...
3,(edit conflict with Piotrus) I agree with the ...,Thanks to Lysy for shortening and copy editing...,0,(edit conflict with Piotrus) I agree with the...
4,The larger box displays fine in Mozilla Firefo...,I like the additional info in the info box. It...,0,The larger box displays fine in Mozilla Firef...


In [ ]:
df_t.head()

,parent,text,label,text_gen
0,I also feel its important for StuRat to have h...,"BTW, I see no ""attack"" here. What I see is som...",0,"BTW, I see no ""attack"" here. What I see is so..."
1,A main reason this project was started was to ...,I realise... I just want to urge caution and a...,0,I realise... I just wanted to make sure you h...
2,"I'd go for that. (By the way, not all deletion...",I should have stressed that this should be a t...,0,I should have stressed that this should be a ...
3,(edit conflict with Piotrus) I agree with the ...,Thanks to Lysy for shortening and copy editing...,0,Thanks to Lysy for suggesting this. They're b...
4,The larger box displays fine in Mozilla Firefo...,I like the additional info in the info box. It...,0,I like the additional info in the info box. I...


In [ ]:
sum([ p!=t for p,t in zip(df_p.parent, df_t.parent)]) # identical
sum([ p!=t for p,t in zip(df_p.label, df_t.label)]) 

0

In [ ]:
configs

{'path': {'list_csv': [PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/gc_clean.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/gc_clean_ner.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/gab_reddit.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/gab_reddit_ner.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/parent_gen_data.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/text_gen_data.csv')],
  'path_colab_data': PosixPath('dataset'),
  'path_drive': PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset')}}

In [ ]:
df_merged = df_p.merge(df_t[['parent','text_gen']], how='inner', on=['parent'])

In [ ]:
df_merged.to_csv(configs.path['path_drive'] / 'generated_dataset.csv')

## Combination of Generated Text Data
- GTP-neo를 통해 생성된 parent, text를 조합.
- 각 개별 데이터 조합으로 Electra model trining.
- Kaggle data로 일반화 성능 평가
  1. parent(O) - text(O)
  2. parent(G) - text(O)
  3. parent(O) - text(G)
  4. parent(G) - text(G)
  5. parent(O+G) - text(O+G)
  - G : generated, O: original

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

from pathlib import Path
import numpy as np
import pandas as pd


class MyConfig(dict):
  def __getattr__(self, name): return self[name]
  def __setattr__(self, name, value): self[name] = value


configs = MyConfig({'path':{
    'path_drive' : Path('/content/gdrive/MyDrive/toxic_nlp'),
    'path_colab_data' : Path('./dataset')
    }
})

configs.path['dataset'] = [f for f in (configs.path['path_drive'] / 'dataset').glob('*') if 'generated_dataset.csv' in str(f)][0]
configs.path['list_csv'] = [f for f in (configs.path['path_drive'] / 'dataset').glob('*.csv')]

Mounted at /content/gdrive


In [ ]:
configs

{'path': {'dataset': PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/generated_dataset.csv'),
  'list_csv': [PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/gc_clean.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/gc_clean_ner.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/gab_reddit.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/gab_reddit_ner.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/parent_gen_data.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/text_gen_data.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/parent_gen_data_v2.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/text_gen_data_v2.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/generated_dataset.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/jig_test.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/jig_all.csv')],
  'path_colab_data': PosixPath('dataset'),
  '

In [ ]:
df_g1 = pd.read_csv([f for f in configs.path['list_csv'] if 'parent_gen_data_v2.csv' in str(f)][0])

In [ ]:
df_g1[df_g1['parent'].duplicated()]

,Unnamed: 0,parent,text,label,parent_gen
4183,4185,How about ?,"Looks legit, but I thought we generally avoide...",0,How about?
4341,4343,See this video of a Wikimania address by resea...,m:Global message delivery/Spam,0,See this video of a different day where a cop...
4701,4703,Sounds good to me.,Im glad to hear it. My intent really isn't to ...,0,Sounds like a lot of work to me.
5346,5348,— Martin ( ·,"Sorry about that. Thanks for fixing it up, guys.",0,—
6054,6057,Thank you.,And it's apparent that I made a repetitive err...,0,Thank you. I will remove it tomorrow because ...
...,...,...,...,...,...
23273,13745,[removed],About the same time you became a retard.,1,[removed]
23375,13847,[deleted],You’re retarded,1,[deleted]
23483,13955,[removed],So asks the nigger kike xD,1,[removed]
23549,14021,[deleted],Your mom is a cunt. You should stop talking to...,0,[deleted]


In [ ]:
df_g1[df_g1['parent'].duplicated()].iloc[-2,2]

'Your mom is a cunt. You should stop talking to her.'

In [ ]:
df_g2 = pd.read_csv([f for f in configs.path['list_csv'] if 'text_gen_data_v2.csv' in str(f)][0])

In [ ]:
df_g2[df_g2['parent'].duplicated()]

,Unnamed: 0,parent,text,label,text_gen
4183,4185,How about ?,"Looks legit, but I thought we generally avoide...",0,"Looks legit, but I thought we generally avoid..."
4341,4343,See this video of a Wikimania address by resea...,m:Global message delivery/Spam,0,m:Global message delivery/Spam
4701,4703,Sounds good to me.,Im glad to hear it. My intent really isn't to ...,0,Im glad to hear it. My intent here is to prom...
5346,5348,— Martin ( ·,"Sorry about that. Thanks for fixing it up, guys.",0,Sorry about that! ) I also updated the descri...
6054,6057,Thank you.,And it's apparent that I made a repetitive err...,0,And it's apparent from his face that he's an ...
...,...,...,...,...,...
23273,13745,[removed],About the same time you became a retard.,1,"About the author, not much. -"
23375,13847,[deleted],You’re retarded,1,You’re retarded.
23483,13955,[removed],So asks the nigger kike xD,1,"So now you claim to be a liberal, retard. You..."
23549,14021,[deleted],Your mom is a cunt. You should stop talking to...,0,Your mom is a cunt


In [ ]:
df_merged = pd.concat([df_g1,df_g2], axis=1)

In [ ]:
sum([i==j for i,j in zip(df_merged.iloc[:,1], df_merged.iloc[:,6])]) # identical

23674

In [ ]:
df_merged.head()

,Unnamed: 0,parent,text,label,parent_gen,Unnamed: 0,parent,text,label,text_gen
0,0,I also feel its important for StuRat to have h...,"BTW, I see no ""attack"" here. What I see is som...",0,I also feel its important for StuRat to have ...,0,I also feel its important for StuRat to have h...,"BTW, I see no ""attack"" here. What I see is som...",0,"BTW, I see no ""attack"" here. What I see is so..."
1,1,A main reason this project was started was to ...,I realise... I just want to urge caution and a...,0,A main reason this project was started was to...,1,A main reason this project was started was to ...,I realise... I just want to urge caution and a...,0,I realise... I just wanted to make sure you h...
2,2,"I'd go for that. (By the way, not all deletion...",I should have stressed that this should be a t...,0,I'd go for that. A lot of the most common are...,2,"I'd go for that. (By the way, not all deletion...",I should have stressed that this should be a t...,0,I should have stressed that this should be a ...
3,3,(edit conflict with Piotrus) I agree with the ...,Thanks to Lysy for shortening and copy editing...,0,(edit conflict with Piotrus) I agree with the...,3,(edit conflict with Piotrus) I agree with the ...,Thanks to Lysy for shortening and copy editing...,0,Thanks to Lysy for suggesting this. They're b...
4,4,The larger box displays fine in Mozilla Firefo...,I like the additional info in the info box. It...,0,The larger box displays fine in Mozilla Firef...,4,The larger box displays fine in Mozilla Firefo...,I like the additional info in the info box. It...,0,I like the additional info in the info box. I...


In [ ]:
df_merged = df_merged.iloc[:,[1,2,3,4,9]]
df_merged

In [ ]:
df_gen = df_merged[['text_gen','parent_gen','label']].rename({'parent_gen':'parent','text_gen':'text'}, axis=1)
df_gen

,text,parent,label
0,"BTW, I see no ""attack"" here. What I see is so...",I also feel its important for StuRat to have ...,0
1,I realise... I just wanted to make sure you h...,A main reason this project was started was to...,0
2,I should have stressed that this should be a ...,I'd go for that. A lot of the most common are...,0
3,Thanks to Lysy for suggesting this. They're b...,(edit conflict with Piotrus) I agree with the...,0
4,I like the additional info in the info box. I...,The larger box displays fine in Mozilla Firef...,0
...,...,...,...
23669,I guess 'cause there are some checks keeping ...,"That's almost too ridiculous to be true, but ...",1
23670,"Don’t cry too hard into your waifu, faggot.",You sound triggered. Did you get the link to ...,1
23671,The media will cover for you. You've just edi...,Here's what's going to happen: Broward is mag...,0
23672,"I dont know exactly how you feel, of course, ...","Yeah tone ""ghost"" from the victory inn is sti...",1


In [ ]:
df_ori = df_merged[['text','parent','label']]
df_ori

,text,parent,label
0,"BTW, I see no ""attack"" here. What I see is som...",I also feel its important for StuRat to have h...,0
1,I realise... I just want to urge caution and a...,A main reason this project was started was to ...,0
2,I should have stressed that this should be a t...,"I'd go for that. (By the way, not all deletion...",0
3,Thanks to Lysy for shortening and copy editing...,(edit conflict with Piotrus) I agree with the ...,0
4,I like the additional info in the info box. It...,The larger box displays fine in Mozilla Firefo...,0
...,...,...,...
23669,I guess 'cause there are some checks keeping t...,"That's almost too ridiculous to be true, but I...",1
23670,"Don’t cry too hard into your waifu, faggot!",You sound triggered. Did your sippy cup run ou...,1
23671,The media will cover for the Dems and call rep...,Here's what's going to happen: Broward is magi...,0
23672,"I dont know exactly how you feel, of course, b...","Yeah tone ""ghost"" from the victory inn is stil...",1


1. text(Ori) - parent(ori)

In [ ]:
condition = 'ori-ori'
df_ori.to_csv(configs.path['path_drive'] / 'dataset' / f'combination-{condition}.csv')

2. text(Ori) - parent(Gan)

In [ ]:
condition = 'ori-gan'
pd.concat([ df_ori['text'],df_gen[['parent','label']] ], axis=1).to_csv(configs.path['path_drive'] / 'dataset' / f'combination-{condition}.csv')

3. text(Gan) - parent(Ori)

In [ ]:
# text gen, parant ori
condition = 'gan-ori'
pd.concat([ df_gen['text'],df_ori[['parent','label']] ], axis=1).to_csv(configs.path['path_drive'] / 'dataset' / f'combination-{condition}.csv')

4. text(Gan) - parent(Gan)

In [ ]:
condition = 'gan-gan'
df_ori.to_csv(configs.path['path_drive'] / 'dataset' / f'combination-{condition}.csv')

5. text(Ori+Gan) - parent(Ori+Gan)

In [ ]:
condition = 'all-merged'
pd.concat([ df_gen, df_ori ], axis=0, ignore_index=True).to_csv(configs.path['path_drive'] / 'dataset' / f'combination-{condition}.csv')

# Kaggle - jigsaw dataset
### for benchmark
- 훈련된 모델을 평가할 수 있는 맥락을 포함하는 혐오발언 데이터셋이 존재하지 않음.
- Kaggle - jigsaw dataset을 활용, 맥락이 None으로 제공될 때 모델의 분류성능을 측정.
- 모델의 일반화 성능!

In [1]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [ ]:
!ln -s /content/gdrive/My\ Drive/toxic_nlp /toxic_nlp
!ls /toxic_nlp

In [3]:
from pathlib import Path
import numpy as np
import pandas as pd

import shutil
import zipfile

In [23]:
class MyConfig(dict):
  def __getattr__(self, name): return self[name]
  def __setattr__(self, name, value): self[name] = value


configs = MyConfig({'path':{
    'path_drive' : Path('/toxic_nlp'),
    'path_colab_data' : Path('./dataset')
    }
})

# configs.path['dataset'] = [f for f in (configs.path['path_drive'] / 'dataset').glob('*') if 'generated_dataset.csv' in str(f)][0]
configs.path['list_csv'] = [f for f in (configs.path['path_drive'] / 'dataset').glob('*.csv')]


# src = [ f for f in (configs.path['path_drive'] / 'dataset' ).glob('*') if '.DS_' not in str(f) ]
# print(src)


In [25]:
def get_data_csv_path(configs, condition):
  condition = condition
  return [f for f in configs.path['list_csv'] if f'{condition}' in str(f)][0]

In [24]:
configs.path['list_csv']

[PosixPath('/toxic_nlp/dataset/gc_clean.csv'),
 PosixPath('/toxic_nlp/dataset/gc_clean_ner.csv'),
 PosixPath('/toxic_nlp/dataset/gab_reddit.csv'),
 PosixPath('/toxic_nlp/dataset/gab_reddit_ner.csv'),
 PosixPath('/toxic_nlp/dataset/parent_gen_data.csv'),
 PosixPath('/toxic_nlp/dataset/text_gen_data.csv'),
 PosixPath('/toxic_nlp/dataset/parent_gen_data_v2.csv'),
 PosixPath('/toxic_nlp/dataset/text_gen_data_v2.csv'),
 PosixPath('/toxic_nlp/dataset/generated_dataset.csv'),
 PosixPath('/toxic_nlp/dataset/combination-gan-gan.csv'),
 PosixPath('/toxic_nlp/dataset/combination-ori-ori.csv'),
 PosixPath('/toxic_nlp/dataset/combination-ori-gan.csv'),
 PosixPath('/toxic_nlp/dataset/combination-gan-ori.csv'),
 PosixPath('/toxic_nlp/dataset/combination-all-merged.csv'),
 PosixPath('/toxic_nlp/dataset/tweet.csv'),
 PosixPath('/toxic_nlp/dataset/hate_speech18.csv'),
 PosixPath('/toxic_nlp/dataset/jig_test_labels.csv'),
 PosixPath('/toxic_nlp/dataset/jig_train.csv'),
 PosixPath('/toxic_nlp/dataset/jig_

In [30]:
df_jig_train = pd.read_csv(get_data_csv_path(configs, 'jig_train'))
df_jig_test = pd.read_csv(get_data_csv_path(configs, 'jig_test'))
df_jig_test_label = pd.read_csv(get_data_csv_path(configs, 'jig_test_labels'))


In [38]:
df_jig_train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [37]:
df_jig_train.toxic.value_counts()

0    144277
1     15294
Name: toxic, dtype: int64

In [46]:
import re

In [61]:
class Regex_pattern():
  def __init__(self):
    self.emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"u"\U0001F300-\U0001F5FF"u"\U0001F680-\U0001F6FF"u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"u"\U00002702-\U000027B0"u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"u"\U0001f926-\U0001f937"u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"u"\u2600-\u2B55"u"\u200d"u"\u23cf"u"\u23e9"u"\u231a"u"\ufe0f"u"\u3030""]+", flags=re.UNICODE)
    self.comment_number = re.compile("[0-9]+.")
    self.user = re.compile(r"@[a-zA-Z]+")
    self.hash = re.compile(r"#")
    self.unlatter = re.compile("[âð±ïó¾¬ª¯¶¦â¡©§¸©¹¼¥¤ã»ÃÀÁÂÃÄÅ½º¢µ$|¿°«à]")
    self.space = re.compile(r"\s+") # too many space
    
    self.url = re.compile("(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))")
    self.url_1 = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$\-@\.&+:/?=]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    self.url_2 = re.compile('www(?:[a-zA-Z]|[0-9]|[$\-@\.&+:/?=]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    self.url_3 = re.compile('http[s]? : // (?:[a-zA-Z]|[0-9]|[$\-@\.&+:/?=]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    
    self.ip = re.compile('\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}')
    self.article_id = re.compile('d:\d\d\s{0,5}$')
    self.userid = re.compile('\[\[User(.*)\|')

def remove_emoji(string, regex_pattern):

    string = regex_pattern.emoji_pattern.sub(r'', string)
    # string = regex_pattern.comment_number.sub(r'',string)
    string = regex_pattern.url.sub(r'',string)
    # user @ tag는 전체 삭제
    string = regex_pattern.user.sub(r'',string)
    # #tag 해시태그는 #해시 문자만 삭제
    string = regex_pattern.hash.sub(r'',string)
    # string = re.compile(r"#[a-zA-Z]+").sub(r'',string)
    string = regex_pattern.unlatter.sub(r'',string)
    string = regex_pattern.space.sub(' ', string).strip()
    string = regex_pattern.url_1.sub('',string)
    string = regex_pattern.url_2.sub('',string)
    string = regex_pattern.url_3.sub('',string)
    return string

regex_pattern = Regex_pattern()

In [51]:
df = df_jig_train

In [53]:
df['count_sent']=df["comment_text"].apply(lambda x: len(re.findall("\n",str(x)))+1)
#Word count in each comment:
df['count_word']=df["comment_text"].apply(lambda x: len(str(x).split()))
#Unique word count
df['count_unique_word']=df["comment_text"].apply(lambda x: len(set(str(x).split())))
#Letter count
df['count_letters']=df["comment_text"].apply(lambda x: len(str(x)))
#punctuation count
# df["count_punctuations"] =df["comment_text"].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
#upper case words count
df["count_words_upper"] = df["comment_text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
#title case words count
df["count_words_title"] = df["comment_text"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
#Number of stopwords
# df["count_stopwords"] = df["comment_text"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))
#Average length of the words
df["mean_word_len"] = df["comment_text"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))


In [55]:
#derived features
#Word count percent in each comment:
df['word_unique_percent']=df['count_unique_word']*100/df['count_word']

In [57]:
#spammers - comments with less than 40% unique words
spammers=df[df['word_unique_percent']<30]


In [ ]:
# 너무 단순한 한단어, 두단어 밖에 안되고 별 의미없는 복사 & 붙여넣기 문장 삭제
for f in spammers.comment_text:
  print(f)

In [60]:
print(df.shape)
df = df[df['word_unique_percent']>=30]
df.shape

(159571, 16)


(159100, 16)

In [ ]:
df_sample = df_jig_train.sample(100)
for f in df_sample.comment_text:
  # print(repr(f))
  print(remove_emoji(f, regex_pattern))

In [ ]:
df['text_clean'] = df['comment_text'].apply(lambda x: remove_emoji(x, regex_pattern))
df.head()

In [67]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,count_sent,count_word,count_unique_word,count_letters,count_words_upper,count_words_title,mean_word_len,word_unique_percent,text_clean
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,2,43,41,264,2,11,5.162791,95.348837,Explanation Why the edits made under my userna...
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,1,17,17,112,1,3,5.588235,100.000000,D'aww! He matches this background colour I'm s...
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,1,42,39,233,0,2,4.571429,92.857143,"Hey man, I'm really not trying to edit war. It..."
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,5,113,82,622,5,7,4.486726,72.566372,""" More I can't make any real suggestions on im..."
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,1,13,13,67,0,2,4.230769,100.000000,"You, sir, are my hero. Any chance you remember..."


In [72]:
df_jig_clean = df[['text_clean','toxic']].rename({'text_clean':'text','toxic':'label'})
df_jig_clean.head()

df_jig_clean.to_csv(configs.path['path_drive'] / 'dataset' / 'jig_train.csv' )

In [ ]:
configs

In [ ]:
# load train & test csv files
files = [f for f in configs.path['path_colab_data'].glob('*.csv') if '.zip' not in str(f)]
jig_train = pd.read_csv([str(f) for f in files if 'train' in str(f)][0])
jig_test = pd.read_csv([str(f) for f in files if 'test' in str(f)][0])
jig_test_label = pd.read_csv([str(f) for f in files if 'test_label' in str(f)][0])

# Conan set

In [ ]:
!wget https://raw.githubusercontent.com/marcoguerini/CONAN/master/CONAN/CONAN.csv conan.csv

--2021-07-16 06:29:24--  https://raw.githubusercontent.com/marcoguerini/CONAN/master/CONAN/CONAN.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4035168 (3.8M) [text/plain]
Saving to: ‘CONAN.csv’

CONAN.csv           100%[===================>]   3.85M  --.-KB/s    in 0.07s   

2021-07-16 06:29:29 (56.4 MB/s) - ‘CONAN.csv’ saved [4035168/4035168]

--2021-07-16 06:29:29--  http://conan.csv/
Resolving conan.csv (conan.csv)... failed: Name or service not known.
wget: unable to resolve host address ‘conan.csv’
FINISHED --2021-07-16 06:29:29--
Total wall clock time: 4.8s
Downloaded: 1 files, 3.8M in 0.07s (56.4 MB/s)


In [ ]:
df_conan = pd.read_csv('CONAN.csv')

In [ ]:
df_conan.head()

,cn_id,hateSpeech,counterSpeech,hsType,hsSubType,cnType,age,gender,educationLevel
0,ENT1ST0001HS0033CN000021,"According to a recent OFSTED report, a school ...","To be fair, the OFSTED report is more concerne...",Islamophobia,crimes,facts,61.0,male,Bachelor
1,ENT1ST0001HS0033CN000021P1,In Birmingham there is a school where girls an...,"To be fair, the OFSTED report is more concerne...",Islamophobia,crimes,facts,61.0,male,Bachelor
2,ENT1ST0001HS0033CN000021P2,A school in Birningham is still separating gir...,"To be fair, the OFSTED report is more concerne...",Islamophobia,crimes,facts,61.0,male,Bachelor
3,ENT1ST0001HS0033CN000022,"According to a recent OFSTED report, a school ...",While this story is presented as an 'Islamic i...,Islamophobia,crimes,facts,61.0,male,Bachelor
4,ENT1ST0001HS0033CN000022P1,In Birmingham there is a school where girls an...,While this story is presented as an 'Islamic i...,Islamophobia,crimes,facts,61.0,male,Bachelor


In [ ]:
df_conan.hsType.value_counts()

Islamophobia    14988
Name: hsType, dtype: int64

# Tweets hate speech detection dataset


In [1]:
# !pip install transformers
!pip install datasets

In [2]:
!pip install tqdm==4.43.0

In [3]:
from datasets import load_dataset
import pandas as pd

In [4]:
from datasets import load_dataset

dataset = load_dataset('tweets_hate_speech_detection')

Using custom data configuration default


Dataset tweets_hate_speech_detection downloaded and prepared to /root/.cache/huggingface/datasets/tweets_hate_speech_detection/default/0.0.0/3e953745870454cf8ff15cc48097dbb5ff459596e0a089867c2a29cee63984ec. Subsequent calls will reuse this data.


In [5]:
df = pd.DataFrame({'tweet': dataset['train'][:]['tweet'], 'label': dataset['train'][:]['label']})
df.label.value_counts()

0    29720
1     2242
Name: label, dtype: int64

In [8]:
import re

In [60]:
class Regex_pattern():
  def __init__(self):
    self.emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    self.comment_number = re.compile("[0-9]+.")
    self.url = re.compile("(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))")
    self.user = re.compile(r"@[a-zA-Z]+")
    self.hash = re.compile(r"#")
    self.unlatter = re.compile("[âð±ïó¾¬ª¯¶¦â¡©§¸©¹¼¥¤ã»ÃÀÁÂÃÄÅ½º¢µ$|¿°«à]")
    self.space = re.compile(r"\s+") # too many space


def remove_emoji(string, regex_pattern):

    string = regex_pattern.emoji_pattern.sub(r'', string)
    string = regex_pattern.comment_number.sub(r'',string)
    string = regex_pattern.url.sub(r'',string)
    # user @ tag는 전체 삭제
    string = regex_pattern.user.sub(r'',string)
    # #tag 해시태그는 #해시 문자만 삭제
    string = regex_pattern.hash.sub(r'',string)
    # string = re.compile(r"#[a-zA-Z]+").sub(r'',string)
    string = regex_pattern.unlatter.sub(r'',string)
    string = regex_pattern.space.sub(' ', string).strip()
    return string

regex_pattern = Regex_pattern()

In [61]:
df_sample = df.sample(100)

In [ ]:
df_sample = df.sample(100)
for f in df_sample.tweet:
  # print('ori',f)
  f = remove_emoji(f,regex_pattern)
  print('reg',f)

In [72]:
df['tweet'].apply(lambda x: remove_emoji(x, regex_pattern))

0        when a father is dysfunctional and is so selfi...
1        thanks for lyft credit i can't use cause they ...
2                                      bihday your majesty
3        model i love u take with u all the time in ur...
4                       factsguide: society now motivation
                               ...                        
31957     ate isz that youuu?
31958    to see nina turner on the airwaves trying to w...
31959    listening to sad songs on a monday morning otw...
31960    sikh temple vandalised in in calgary, wso cond...
31961                             thank you for you follow
Name: tweet, Length: 31962, dtype: object

In [74]:
df['tweet_clean'] = df['tweet'].apply(lambda x: remove_emoji(x, regex_pattern))
df.head()
df_clean = df[['tweet_clean','label']]
df_clean.head()

,tweet_clean,label
0,when a father is dysfunctional and is so selfi...,0
1,thanks for lyft credit i can't use cause they ...,0
2,bihday your majesty,0
3,model i love u take with u all the time in ur...,0
4,factsguide: society now motivation,0


In [114]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [115]:
!ln -s /content/gdrive/My\ Drive/toxic_nlp /toxic_nlp
!ls /toxic_nlp

 BentoML.ipynb
 BERT_dynamic_quantiztion_tutorial.ipynb
 dataset
 Electra-model
 Electra-model-save
 Electra_with_pair_sentence.ipynb
 Electra_with_pair_sentence_trained_generated_dataset.ipynb
 Electra_with_pair_sentence_trained_vanilla_dataset.ipynb
 Electra_with_pair_Sentence_Training_Eval.ipynb
 fine-tuneWith_transformer_API_tutorial.ipynb
 gpt-neo-checkpoints
'GTP-neo-fine-tuning(parents-text).ipynb'
'GTP-neo-fine-tuning(toxic-text).ipynb'
 Hope-speech-tutorial.ipynb
 Inference_Time.ipynb
 ktrain.ipynb
 Preprocessing_data_and_prepare4training.ipynb
 preprocessing_datasets.ipynb
 pytorch_tutorial.ipynb
 toxic_v1.ipynb
 toxic_v2.ipynb


In [116]:
from pathlib import Path

class MyConfig(dict):
  def __getattr__(self, name): return self[name]
  def __setattr__(self, name, value): self[name] = value


configs = MyConfig({'path_drive': Path('/toxic_nlp')
})
configs.path_drive / 'dataset'



PosixPath('/toxic_nlp/dataset')

In [75]:
df_clean.to_csv(configs.path_drive / 'dataset' / 'tweet.csv')

In [76]:
df_gc = pd.read_csv(configs.path_drive / 'dataset' / 'gc_clean_ner.csv')


In [ ]:
df_gc

# Hate speech 18 from 백인우월주의자 포럼


In [ ]:
!pip install datasets
!pip install tqdm==4.43.0

In [2]:
from datasets import load_dataset
import pandas as pd
import re

In [15]:
df = load_dataset('hate_speech18')
df

Using custom data configuration default
Reusing dataset hate_speech18 (/root/.cache/huggingface/datasets/hate_speech18/default/0.0.0/8033f254483a20d1d10b0f1b56ded1f54326b1617872c537f354a721a6951d47)


DatasetDict({
    train: Dataset({
        features: ['text', 'user_id', 'subforum_id', 'num_contexts', 'label'],
        num_rows: 10944
    })
})

In [16]:
df = pd.DataFrame({'text': df['train']['text'], 
                   'label':df['train']['label'], 
                   'num_contexts':df['train']['num_contexts'] })

df.label.value_counts()

0    9507
1    1196
3     168
2      73
Name: label, dtype: int64

### label
- 0: non toxic
- 1: toxic
- 2: 외국어
- 3: 해당 문장은 toxic 하지 않지만, 맥락이 있으면 toxic (맥락은 제공되지 않음)


In [34]:
df[ ((df.label == 0) | (df.label==1)) ].label.value_counts()
# take 0,1
df = df[ ((df.label == 0) | (df.label==1)) ]

In [54]:
class Regex_pattern():
  def __init__(self):
    self.emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    self.comment_number = re.compile("[0-9]+.")
    self.url = re.compile("(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))")
    self.user = re.compile(r"@[a-zA-Z]+")
    self.hash = re.compile(r"#")
    self.unlatter = re.compile("[âð±ïó¾¬ª¯¶¦â¡©§¸©¹¼¥¤ã»ÃÀÁÂÃÄÅ½º¢µ$|¿°«à]")
    self.space = re.compile(r"\s+") # too many space
    self.url_1 = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$\-@\.&+:/?=]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    self.url_2 = re.compile('www(?:[a-zA-Z]|[0-9]|[$\-@\.&+:/?=]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    self.url_3 = re.compile('http[s]? : // (?:[a-zA-Z]|[0-9]|[$\-@\.&+:/?=]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
# m = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$\-@\.&+:/?=]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', str1)

def remove_emoji(string, regex_pattern):

    string = regex_pattern.emoji_pattern.sub(r'', string)
    # string = regex_pattern.comment_number.sub(r'',string)
    string = regex_pattern.url.sub(r'',string)
    # user @ tag는 전체 삭제
    string = regex_pattern.user.sub(r'',string)
    # #tag 해시태그는 #해시 문자만 삭제
    string = regex_pattern.hash.sub(r'',string)
    # string = re.compile(r"#[a-zA-Z]+").sub(r'',string)
    string = regex_pattern.unlatter.sub(r'',string)
    string = regex_pattern.space.sub(' ', string).strip()
    string = regex_pattern.url_1.sub('',string)
    string = regex_pattern.url_2.sub('',string)
    string = regex_pattern.url_3.sub('',string)
    return string

regex_pattern = Regex_pattern()

In [ ]:
# df_sample = df.sample(100)
for f in df_sample.text:
  print('ori',f)
  f = remove_emoji(f,regex_pattern)
  print('reg',str(f))

In [56]:
df['text_clean'] = df['text'].apply(lambda x: remove_emoji(x, regex_pattern))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df.index.values.tolist()

In [ ]:
# token 2개 이하, 삭제되지 않은 url, 한단어, 공백 제거
less_comment = df.index[df['text_clean'].apply(lambda x : len(x.split(' '))) < 3]
for i in df.loc[less_comment].text_clean:
  print(i)

In [112]:
df = df.drop(less_comment)

In [117]:
df.to_csv(configs.path_drive / 'dataset' / 'hate_speech18.csv')

In [119]:
df.label.value_counts()

0    9057
1    1196
Name: label, dtype: int64

In [123]:
df_clean = df[['text_clean','label']].rename({'text_clean':'text'}, axis=1)
df_clean.to_csv(configs.path_drive / 'dataset' / 'hate_speech18.csv')

# Merge data sets
### 20만건 데이터 셋 모으기

In [1]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [2]:
!ln -s /content/gdrive/My\ Drive/toxic_nlp /toxic_nlp
!ls /toxic_nlp

 BentoML.ipynb
 BERT_dynamic_quantiztion_tutorial.ipynb
 dataset
 Electra-model
 Electra-model-save
 Electra_with_pair_sentence.ipynb
 Electra_with_pair_sentence_trained_generated_dataset.ipynb
 Electra_with_pair_sentence_trained_vanilla_dataset.ipynb
 Electra_with_pair_Sentence_Training_Eval.ipynb
 fine-tuneWith_transformer_API_tutorial.ipynb
 gpt-neo-checkpoints
'GTP-neo-fine-tuning(parents-text).ipynb'
'GTP-neo-fine-tuning(toxic-text).ipynb'
 Hope-speech-tutorial.ipynb
 Inference_Time.ipynb
 ktrain.ipynb
 Preprocessing_data_and_prepare4training.ipynb
 preprocessing_datasets.ipynb
 pytorch_tutorial.ipynb
 toxic_v1.ipynb
 toxic_v2.ipynb


In [3]:
from pathlib import Path
import pandas as pd


In [4]:
def get_data_csv_path(configs, condition):
  condition = condition
  return [f for f in configs.path['list_csv'] if f'{condition}' in str(f)][0]

In [5]:
list_csv = [f for f in Path('/toxic_nlp/dataset').glob('*') if '.csv' in str(f)]

In [6]:
list_csv

[PosixPath('/toxic_nlp/dataset/gc_clean.csv'),
 PosixPath('/toxic_nlp/dataset/gc_clean_ner.csv'),
 PosixPath('/toxic_nlp/dataset/gab_reddit.csv'),
 PosixPath('/toxic_nlp/dataset/gab_reddit_ner.csv'),
 PosixPath('/toxic_nlp/dataset/parent_gen_data.csv'),
 PosixPath('/toxic_nlp/dataset/text_gen_data.csv'),
 PosixPath('/toxic_nlp/dataset/parent_gen_data_v2.csv'),
 PosixPath('/toxic_nlp/dataset/text_gen_data_v2.csv'),
 PosixPath('/toxic_nlp/dataset/generated_dataset.csv'),
 PosixPath('/toxic_nlp/dataset/tweet.csv'),
 PosixPath('/toxic_nlp/dataset/hate_speech18.csv'),
 PosixPath('/toxic_nlp/dataset/jig_train.csv'),
 PosixPath('/toxic_nlp/dataset/merged.csv')]

In [7]:
df_cat_gabreddit = pd.read_csv([f for f in list_csv if 'generated_dataset.csv' in str(f)][0])
df_hate_speech = pd.read_csv([f for f in list_csv if 'hate_speech18.csv' in str(f)][0])
df_jig_train = pd.read_csv([f for f in list_csv if 'jig_train.csv' in str(f)][0])
df_tweet = pd.read_csv([f for f in list_csv if 'tweet.csv' in str(f)][0])

In [8]:
df_cat_gabreddit['src'] = 0
df_cat_gabreddit.head()

,Unnamed: 0,parent,text,label,parent_gen,text_gen,src
0,0,I also feel its important for StuRat to have h...,"BTW, I see no ""attack"" here. What I see is som...",0,I also feel its important for StuRat to have ...,"BTW, I see no ""attack"" here. What I see is so...",0
1,1,A main reason this project was started was to ...,I realise... I just want to urge caution and a...,0,A main reason this project was started was to...,I realise... I just wanted to make sure you h...,0
2,2,"I'd go for that. (By the way, not all deletion...",I should have stressed that this should be a t...,0,I'd go for that. A lot of the most common are...,I should have stressed that this should be a ...,0
3,3,(edit conflict with Piotrus) I agree with the ...,Thanks to Lysy for shortening and copy editing...,0,(edit conflict with Piotrus) I agree with the...,Thanks to Lysy for suggesting this. They're b...,0
4,4,The larger box displays fine in Mozilla Firefo...,I like the additional info in the info box. It...,0,The larger box displays fine in Mozilla Firef...,I like the additional info in the info box. I...,0


In [9]:
sum(df_cat_gabreddit.duplicated())

0

In [10]:
df_hate_speech['src'] = 2
df_hate_speech.head()

,Unnamed: 0,text,label,src
0,0,"As of March 13th , 2014 , the booklet had been...",0,2
1,1,In order to help increase the booklets downloa...,0,2
2,2,( Simply copy and paste the following text int...,0,2
3,3,Click below for a FREE download of a colorfull...,1,2
4,4,Click on the `` DOWNLOAD ( 7.42 MB ) '' green ...,0,2


In [11]:
sum(df_hate_speech.duplicated())

0

In [12]:
df_jig_train.rename({'text_clean':'text','toxic':'label'}, inplace=True, axis=1)
df_jig_train['src'] = 1
df_jig_train.head()

,Unnamed: 0,text,label,src
0,0,Explanation Why the edits made under my userna...,0,1
1,1,D'aww! He matches this background colour I'm s...,0,1
2,2,"Hey man, I'm really not trying to edit war. It...",0,1
3,3,""" More I can't make any real suggestions on im...",0,1
4,4,"You, sir, are my hero. Any chance you remember...",0,1


In [13]:
sum(df_jig_train.duplicated())

0

In [14]:
df_tweet.rename({'tweet_clean':'text'}, axis=1, inplace=True)
df_tweet['src'] = 3
df_tweet.head()

,Unnamed: 0,text,label,src
0,0,when a father is dysfunctional and is so selfi...,0,3
1,1,thanks for lyft credit i can't use cause they ...,0,3
2,2,bihday your majesty,0,3
3,3,model i love u take with u all the time in ur...,0,3
4,4,factsguide: society now motivation,0,3


In [15]:
sum(df_tweet.duplicated())

0

In [29]:
df_merged = pd.concat([df_cat_gabreddit,df_tweet, df_jig_train, df_hate_speech], axis = 0, ignore_index=True)
df_merged.head()

,Unnamed: 0,parent,text,label,parent_gen,text_gen,src
0,0,I also feel its important for StuRat to have h...,"BTW, I see no ""attack"" here. What I see is som...",0,I also feel its important for StuRat to have ...,"BTW, I see no ""attack"" here. What I see is so...",0
1,1,A main reason this project was started was to ...,I realise... I just want to urge caution and a...,0,A main reason this project was started was to...,I realise... I just wanted to make sure you h...,0
2,2,"I'd go for that. (By the way, not all deletion...",I should have stressed that this should be a t...,0,I'd go for that. A lot of the most common are...,I should have stressed that this should be a ...,0
3,3,(edit conflict with Piotrus) I agree with the ...,Thanks to Lysy for shortening and copy editing...,0,(edit conflict with Piotrus) I agree with the...,Thanks to Lysy for suggesting this. They're b...,0
4,4,The larger box displays fine in Mozilla Firefo...,I like the additional info in the info box. It...,0,The larger box displays fine in Mozilla Firef...,I like the additional info in the info box. I...,0


In [17]:
df_merged.reset_index(inplace=True)
df_merged.drop(columns=['Unnamed: 0','index'], axis=1, inplace=True)

In [31]:
df_merged[df_merged.duplicated()]

,Unnamed: 0,parent,text,label,parent_gen,text_gen,src


In [32]:
df_merged.to_csv('/toxic_nlp/dataset/merged.csv')

# Split test set with same ratio
- 데이터 출처마다 sample size 차이가 크게 나기 때문에 데이터 출처와 label의 비율을 맞추면서 test set 구성
- generated text 따로 파일로 만들어 둘 것


In [71]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

!ln -s /content/gdrive/My\ Drive/toxic_nlp /toxic_nlp
!ls /toxic_nlp

from pathlib import Path
import pandas as pd


def get_data_csv_path(configs, condition):
  condition = condition
  return [f for f in configs.path['list_csv'] if f'{condition}' in str(f)][0]



Mounted at /content/gdrive
 BentoML.ipynb
 BERT_dynamic_quantiztion_tutorial.ipynb
'Copy of Electra_with_pair_Sentence_Training_Eval.ipynb'
 dataset
 Electra-model
 Electra-model-save
 Electra_with_pair_sentence.ipynb
 Electra_with_pair_sentence_trained_generated_dataset.ipynb
 Electra_with_pair_sentence_trained_vanilla_dataset.ipynb
 Electra_with_pair_Sentence_Training_Eval.ipynb
 fine-tuneWith_transformer_API_tutorial.ipynb
 gpt-neo-checkpoints
'GTP-neo-fine-tuning(parents-text).ipynb'
'GTP-neo-fine-tuning(toxic-text).ipynb'
 Hope-speech-tutorial.ipynb
 Inference_Time.ipynb
 ktrain.ipynb
 Preprocessing_data_and_prepare4training.ipynb
 preprocessing_datasets.ipynb
 pytorch_tutorial.ipynb
 toxic_nlp
 toxic_v1.ipynb
 toxic_v2.ipynb


In [72]:
list_csv = [f for f in Path('/toxic_nlp/dataset').glob('*') if '.csv' in str(f)]
list_csv
df = pd.read_csv([f for f in list_csv if 'merged.csv' in str(f)][0])

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


- 0: gab-reddit
- 1: jig
- 2: stormfront
- 3: tweet

In [73]:
df.src.value_counts()

1    159100
3     31962
0     29758
2     10253
Name: src, dtype: int64

In [74]:
df.drop(['Unnamed: 0','Unnamed: 0.1'], axis=1, inplace=True)

In [75]:
df.head()

,parent,text,label,parent_gen,text_gen,src
0,I also feel its important for StuRat to have h...,"BTW, I see no ""attack"" here. What I see is som...",0,I also feel its important for StuRat to have ...,"BTW, I see no ""attack"" here. What I see is so...",0
1,A main reason this project was started was to ...,I realise... I just want to urge caution and a...,0,A main reason this project was started was to...,I realise... I just wanted to make sure you h...,0
2,"I'd go for that. (By the way, not all deletion...",I should have stressed that this should be a t...,0,I'd go for that. A lot of the most common are...,I should have stressed that this should be a ...,0
3,(edit conflict with Piotrus) I agree with the ...,Thanks to Lysy for shortening and copy editing...,0,(edit conflict with Piotrus) I agree with the...,Thanks to Lysy for suggesting this. They're b...,0
4,The larger box displays fine in Mozilla Firefo...,I like the additional info in the info box. It...,0,The larger box displays fine in Mozilla Firef...,I like the additional info in the info box. I...,0


In [76]:
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
df_train = df.copy()

In [45]:
df[df.src == 1].reset_index()

,index,parent,text,label,parent_gen,text_gen,src
0,61720,NaN,Explanation Why the edits made under my userna...,0,NaN,NaN,1
1,61721,NaN,D'aww! He matches this background colour I'm s...,0,NaN,NaN,1
2,61722,NaN,"Hey man, I'm really not trying to edit war. It...",0,NaN,NaN,1
3,61723,NaN,""" More I can't make any real suggestions on im...",0,NaN,NaN,1
4,61724,NaN,"You, sir, are my hero. Any chance you remember...",0,NaN,NaN,1
...,...,...,...,...,...,...,...
159095,220815,NaN,""":::::And for the second time of asking, when ...",0,NaN,NaN,1
159096,220816,NaN,You should be ashamed of yourself That is a ho...,0,NaN,NaN,1
159097,220817,NaN,"Spitzer Umm, theres no actual article for pros...",0,NaN,NaN,1
159098,220818,NaN,And it looks like it was actually you who put ...,0,NaN,NaN,1


In [77]:
def data_loader(df, condition):
  return df[df.src==condition].reset_index(), df[df.src == condition].reset_index()['label']


In [54]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.15, random_state=42)
train_sets = []
test_sets = []
for i in [0,1,2,3]:
  dataset, label = data_loader(df_train, i)
  for train_idx, val_idx in split.split( dataset, label):
    train_sets.append(dataset.loc[train_idx]) 
    test_sets.append(dataset.loc[val_idx])


In [61]:
for i in [0,1,2,3]:
  print(train_sets[i].src.value_counts())
  print(test_sets[i].label.value_counts())

0    25294
Name: src, dtype: int64
1    2590
0    1874
Name: label, dtype: int64
1    135235
Name: src, dtype: int64
0    21621
1     2244
Name: label, dtype: int64
2    8715
Name: src, dtype: int64
0    1359
1     179
Name: label, dtype: int64
3    27167
Name: src, dtype: int64
0    4459
1     336
Name: label, dtype: int64


In [59]:
df_train_merged = pd.concat([train_sets[0],train_sets[1],train_sets[2],train_sets[3]], ignore_index=True)
df_tset_merged = pd.concat([test_sets[0],test_sets[1],test_sets[2],test_sets[3]], ignore_index=True)

In [60]:
df_train_merged.to_csv('/toxic_nlp/dataset/exp_v1_train.csv')
df_tset_merged.to_csv('/toxic_nlp/dataset/exp_v1_test.csv')

In [62]:
df_train_merged.head()

,index,parent,text,label,parent_gen,text_gen,src
0,23973,RT _Sailer: Speaking of witchcraft (as Vox doe...,call your nigger witch doctor white boy..maybe...,1,RT _Sailer: Speaking of witchcraft (as Vox do...,call your nigger lover a faggot,0
1,14616,"Robert, how is your moral fagging going? When ...",Speaking ones mind.. like how evil the Altrigh...,1,"Robert, how is your moral fagging going? If y...",suck a nigger cock and expect a nigger cock i...,0
2,2487,BTW do we need the co-ords at the top right an...,"Ah, so that's what goes in there! The ones at ...",0,BTW do we intend to make the edit war a non-c...,"Ah, so that's what goes in there...",0
3,7672,What time is meant? Coming together is apparen...,in the Act of Kreva has a Latin word 'aplicare...,0,What time is meant? Coming together and getti...,in the Act of Kreva has a Latin word 'aplicar...,0
4,8157,lol! one diff is sufficient to over-write the ...,I said why I restored and you did the same by ...,0,lol! one diff is sufficient touzzy. Faggot.,I said why I restored your edit in full. Howe...,0


In [67]:
df_gan = df_train_merged[['parent_gen','text_gen','label']].rename(columns={'parent_gen':'parent','text_gen':'text'}).dropna()

In [70]:
pd.concat([df_train_merged,df_gan], axis=0).to_csv('/toxic_nlp/dataset/exp_v1_gan_merged.csv')

In [ ]:
df_gan_merged = pd.read_csv('/toxic_nlp/dataset/exp_v1_gan_merged.csv', index_col=0)
df_gan_merged[df_gan_merged.src==0]

In [94]:
df_gan_merged.shape

(221705, 7)

In [96]:
df_gan_ori = df_gan_merged[df_gan_merged.src==0][['parent_gen','text','label']].rename(columns={'parent_gen':'parent'})
# df_gan_ori
# ori-ori + gan_gan + gan_ori condition. 
pd.concat([df_gan_merged, df_gan_ori], axis=0).to_csv('/toxic_nlp/dataset/exp_v2_gan_merged.csv')